make sure that I did the implementation correctly and can output loss in the same way as before, against <https://github.com/leelabcnbc/thesis-proposal-yimeng/blob/master/results_ipynb_blocks/population_neuron_fitting/debug/debug_maskcnn_v1_loss_PyTorch_only_rewrite.ipynb>

In [1]:
import numpy as np
import torch
from torch.autograd import Variable
from torch import Tensor
from torch.backends import cudnn
cudnn.benchmark = True
cudnn.enabled = True
from torch.nn import MSELoss
from torch import nn

In [2]:
import os.path
import os
from collections import OrderedDict

In [3]:
from maskcnn import cnn_legacy
from tang_jcompneuro import cnn

In [4]:
load_dir = '/home/yimengzh/data2/thesis-proposal-yimeng/results_ipynb_blocks/population_neuron_fitting/debug/tmp/TF/v1_loss'

In [5]:
import pickle
with open(os.path.join(load_dir, 'init_weights.pickle'), 'rb') as f_out:
    init_data = pickle.load(f_out)
    
with open(os.path.join(load_dir, 'losses.pickle'), 'rb') as f_out:
    stat_all_batches_ref = pickle.load(f_out)

In [6]:
stat_all_batches_ref['res_batch'].shape

(20, 103)

In [7]:
stat_all_batches_ref.keys()

dict_keys(['imgs_batch', 'res_batch', 'prediction', 'poisson', 'readout_reg', 'total_loss'])

In [8]:
model_old = cnn_legacy.MaskCNNV1(n=stat_all_batches_ref['res_batch'].shape[1], init_dict_tf=init_data)
model_old = model_old.cuda()
model_old = model_old.train()

In [9]:
from maskcnn.cnn_aux import (get_loss, get_maskcnn_v1_arch_config, v1_maskcnn_generator,
                             get_maskcnn_v1_opt_config)

In [10]:
model_rewritten = cnn.CNN(get_maskcnn_v1_arch_config(factored_constraint='abs'),
    # get_maskcnn_v1_arch_config(factored_constraint=None),  # this would fail
                          v1_maskcnn_generator(),
                  input_size=31, n=stat_all_batches_ref['res_batch'].shape[1], seed=0)
model_rewritten = model_rewritten.cuda()
model_rewritten = model_rewritten.train()

In [11]:
model_rewritten

CNN(
  (conv): Sequential(
    (conv0): Conv2d(1, 48, kernel_size=(13, 13), stride=(1, 1), bias=False)
    (bn0): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True)
    (act0): Softplus(beta=1, threshold=20)
    (conv1): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True)
    (act1): Softplus(beta=1, threshold=20)
    (conv2): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True)
  )
  (fc): Sequential(
    (fc): FactoredLinear2D(
    )
  )
  (final_act): Softplus(beta=1, threshold=20)
)

In [12]:
def compare_two_models_and_transfer(model1, model2):
    
    model_1_dict = dict()
    for x, y in model1.named_parameters():
        print(x, y.size())
        model_1_dict[x] = y
    model_2_dict = dict()
    for x, y in model2.named_parameters():
        print(x, y.size())
        model_2_dict[x] = y
    mapping = {
        'fc.fc.weight_spatial': 'factored_fc_2d.weight_spatial',
        'fc.fc.weight_feature': 'factored_fc_2d.weight_feature',
        'fc.fc.bias': 'factored_fc_2d.bias',
    }
    
    for x, y in model_2_dict.items():
        print(x)
        y2 = model_1_dict[mapping.get(x, x)]
        assert y.size() == y2.size()
        y.data[...] = y2.data
        
    
        
compare_two_models_and_transfer(model_old, model_rewritten)
del model_old

conv.conv0.weight torch.Size([48, 1, 13, 13])
conv.bn0.weight torch.Size([48])
conv.bn0.bias torch.Size([48])
conv.conv1.weight torch.Size([48, 48, 3, 3])
conv.bn1.weight torch.Size([48])
conv.bn1.bias torch.Size([48])
conv.conv2.weight torch.Size([48, 48, 3, 3])
conv.bn2.weight torch.Size([48])
conv.bn2.bias torch.Size([48])
factored_fc_2d.weight_spatial torch.Size([103, 19, 19])
factored_fc_2d.weight_feature torch.Size([103, 48])
factored_fc_2d.bias torch.Size([103])
conv.conv0.weight torch.Size([48, 1, 13, 13])
conv.bn0.weight torch.Size([48])
conv.bn0.bias torch.Size([48])
conv.conv1.weight torch.Size([48, 48, 3, 3])
conv.bn1.weight torch.Size([48])
conv.bn1.bias torch.Size([48])
conv.conv2.weight torch.Size([48, 48, 3, 3])
conv.bn2.weight torch.Size([48])
conv.bn2.bias torch.Size([48])
fc.fc.weight_spatial torch.Size([103, 19, 19])
fc.fc.weight_feature torch.Size([103, 48])
fc.fc.bias torch.Size([103])
conv.conv0.weight
conv.bn0.weight
conv.bn0.bias
conv.conv1.weight
conv.bn1.weig

In [13]:

compute_loss = get_loss(get_maskcnn_v1_opt_config(),model_rewritten, 
                        return_dict=True)


X_batch = stat_all_batches_ref['imgs_batch'].transpose((0,3,1,2))
Y_batch = stat_all_batches_ref['res_batch']
Y_batch = Variable(Tensor(Y_batch).cuda())
Y_this_batch = model_rewritten(Variable(Tensor(X_batch).cuda()))

# compute loss.
stat_all_batches = {
    'prediction': Y_this_batch.data.cpu().numpy(),
#     'conv': bn_this_batch,
}
stat_all_batches.update(compute_loss(Y_this_batch, Y_batch, model_rewritten))
for x_, y_ in stat_all_batches.items():
    print(x_, type(y_))
    print(x_, y_.size)
    if y_.size == 1:
        print(y_)

prediction <class 'numpy.ndarray'>
prediction 2060
group_sparsity <class 'numpy.float32'>
group_sparsity 1
5.96147
smooth_sparsity <class 'numpy.float32'>
smooth_sparsity 1
14.4936
readout_reg <class 'numpy.float32'>
readout_reg 1
1.86128
poisson <class 'numpy.float32'>
poisson 1
91.4548
total_loss <class 'numpy.float32'>
total_loss 1
113.771


In [14]:
from scipy.stats import pearsonr
from numpy.linalg import norm

def compare_one_case(x1, x2, title=None):
    if title is not None:
        print(title, end=' ')
    assert x1.dtype == x2.dtype == np.float32
    assert x1.shape == x2.shape
    print(abs(x1-x2).max(), pearsonr(x1.ravel(), x2.ravel()))
    assert(norm(x1.ravel()-x2.ravel())/norm(x2.ravel())) < 1e-5
    


def compare_batches():
    x1 = stat_all_batches
    x2 = stat_all_batches_ref
    Y_x1, Y_x2 = x1['prediction'], x2['prediction']
    compare_one_case(Y_x1, Y_x2, 'prediction')
#         bn_x1, bn_x2 = x1['conv'], x2['conv']
#         compare_one_case(bn_x1, bn_x2, 'conv')

    for k in ('poisson', 'readout_reg',
              'total_loss'
             ):
        print(k, x1[k], x2[k], abs(x1[k]-x2[k]))
        assert abs(x1[k]-x2[k]) < 1e-5
        
compare_batches()

prediction 1.49012e-07 (1.0, 0.0)
poisson 91.4548 91.4548 0.0
readout_reg 1.86128 1.86128 0.0
total_loss 113.771 113.771 0.0
